# Extract, Transform & Load

Import libraries.

In [1]:
import pandas as pd

Load CSV.

In [7]:
df_traffic_accidents = pd.read_csv("../data/raw/atus_anual_csv/conjunto_de_datos/atus_anual_2023.csv")

In [8]:
df_traffic_accidents.info

<bound method DataFrame.info of         COBERTURA  ID_ENTIDAD  ID_MUNICIPIO  ANIO  MES  ID_HORA  ID_MINUTO  \
0       Municipal           1             1  2023    1        0          0   
1       Municipal           1             1  2023    1        0          0   
2       Municipal           1             1  2023    1        2         20   
3       Municipal           1             1  2023    1        3         20   
4       Municipal           1             1  2023    1        6          0   
...           ...         ...           ...   ...  ...      ...        ...   
396623  Municipal          32            56  2023   12       14         45   
396624  Municipal          32            56  2023   12       21         40   
396625  Municipal          32            56  2023   12       14         30   
396626  Municipal          32            56  2023   12        0          0   
396627  Municipal          32            56  2023   12       18          0   

        ID_DIA  DIASEMANA      

In [9]:
df_traffic_accidents.columns

Index(['COBERTURA', 'ID_ENTIDAD', 'ID_MUNICIPIO', 'ANIO', 'MES', 'ID_HORA',
       'ID_MINUTO', 'ID_DIA', 'DIASEMANA', 'URBANA', 'SUBURBANA', 'TIPACCID',
       'AUTOMOVIL', 'CAMPASAJ', 'MICROBUS', 'PASCAMION', 'OMNIBUS', 'TRANVIA',
       'CAMIONETA', 'CAMION', 'TRACTOR', 'FERROCARRI', 'MOTOCICLET',
       'BICICLETA', 'OTROVEHIC', 'CAUSAACCI', 'CAPAROD', 'SEXO', 'ALIENTO',
       'CINTURON', 'ID_EDAD', 'CONDMUERTO', 'CONDHERIDO', 'PASAMUERTO',
       'PASAHERIDO', 'PEATMUERTO', 'PEATHERIDO', 'CICLMUERTO', 'CICLHERIDO',
       'OTROMUERTO', 'OTROHERIDO', 'NEMUERTO', 'NEHERIDO', 'CLASACC',
       'ESTATUS'],
      dtype='object')

In [10]:
df_traffic_accidents.dtypes

COBERTURA       object
ID_ENTIDAD       int64
ID_MUNICIPIO     int64
ANIO             int64
MES              int64
ID_HORA          int64
ID_MINUTO        int64
ID_DIA           int64
DIASEMANA       object
URBANA          object
SUBURBANA       object
TIPACCID        object
AUTOMOVIL        int64
CAMPASAJ         int64
MICROBUS         int64
PASCAMION        int64
OMNIBUS          int64
TRANVIA          int64
CAMIONETA        int64
CAMION           int64
TRACTOR          int64
FERROCARRI       int64
MOTOCICLET       int64
BICICLETA        int64
OTROVEHIC        int64
CAUSAACCI       object
CAPAROD         object
SEXO            object
ALIENTO         object
CINTURON        object
ID_EDAD          int64
CONDMUERTO       int64
CONDHERIDO       int64
PASAMUERTO       int64
PASAHERIDO       int64
PEATMUERTO       int64
PEATHERIDO       int64
CICLMUERTO       int64
CICLHERIDO       int64
OTROMUERTO       int64
OTROHERIDO       int64
NEMUERTO         int64
NEHERIDO         int64
CLASACC    

Create a date column.